<a href="https://colab.research.google.com/github/ailab-nda/NLP/blob/main/OpenNMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenMNT-py

## 環境設定

In [ ]:
%%bash
git clone https://github.com/OpenNMT/OpenNMT-py.git
cd OpenNMT-py
pip install -e .

### 実行

データの準備（英語ードイツ語）

In [ ]:
%%bash
cd /content/OpenNMT-py/
wget https://s3.amazonaws.com/opennmt-trainingdata/toy-ende.tar.gz
tar xf toy-ende.tar.gz

設定ファイルの作成

In [ ]:
%%bash
cat << EOF > /content/OpenNMT-py/toy_en_de.yaml
## Where the samples will be written
save_data: /content/OpenNMT-py/toy-ende/run/example
## Where the vocab(s) will be written
src_vocab: /content/OpenNMT-py/toy-ende/run/example.vocab.src
tgt_vocab: /content/OpenNMT-py/toy-ende/run/example.vocab.tgt
# Prevent overwriting existing files in the folder
overwrite: False

# Corpus opts:
data:
    corpus_1:
        path_src: /content/OpenNMT-py/toy-ende/src-train.txt
        path_tgt: /content/OpenNMT-py/toy-ende/tgt-train.txt
    valid:
        path_src: /content/OpenNMT-py/toy-ende/src-val.txt
        path_tgt: /content/OpenNMT-py/toy-ende/tgt-val.txt
EOF

In [ ]:
%%bash
cd /content/OpenNMT-py/
onmt_build_vocab -config /content/OpenNMT-py/toy_en_de.yaml -n_sample 10000

In [ ]:
%%bash
cat << EOF >> /content/OpenNMT-py/toy_en_de.yaml
# Vocabulary files that were just created
src_vocab: /content/OpenNMT-py/toy-ende/run/example.vocab.src
tgt_vocab: /content/OpenNMT-py/toy-ende/run/example.vocab.tgt

# Train on a single GPU
world_size: 1
gpu_ranks: [0]

# Where to save the checkpoints
save_model: /content/OpenNMT-py/toy-ende/run/model
save_checkpoint_steps: 500
train_steps: 1000
valid_steps: 500
EOF

In [ ]:
%%bash
cd /content/OpenNMT-py/
onmt_train -config /content/OpenNMT-py/toy_en_de.yaml

In [ ]:
onmt_translate -model toy-ende/run/model_step_1000.pt -src toy-ende/src-test.txt -output toy-ende/pred_1000.txt -gpu 0 -verbose

In [ ]:
!ls /content/OpenNMT-py/toy-ende/run
from google.colab import files
path = "/content/OpenNMT-py/toy-ende/run/"

files.download(path + "example.vocab.src")
files.download(path + "example.vocab.tgt")
files.download(path + "model_step_1000.pt")
files.download(path + "model_step_500.pt")